**AI-Teaching Assistant - Project**

In [3]:
import fitz  
doc = fitz.open("Excerpts - Arendt - The Human Condition.pdf")
text = ""
for page in doc:
    text += page.get_text()

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.create_documents([text])

In [5]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
texts = [doc.page_content for doc in docs]

doc_embeddings = embeddings.embed_documents(texts)

C:\Users\HP\AppData\Local\Temp\ipykernel_338460\2310884709.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [6]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(docs, embeddings)

In [7]:
retriever = db.as_retriever(search_type="similarity", k=3)
relevant_docs = retriever.get_relevant_documents("What is labor for Arendt?")

C:\Users\HP\AppData\Local\Temp\ipykernel_338460\3270450041.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents("What is labor for Arendt?")


In [8]:
from langchain.chains import RetrievalQA
import os
from huggingface_hub import InferenceClient

os.environ["HF_TOKEN"] = "hf_"
query = "What is labor for Arendt?"

context = "\n\n".join(doc.page_content for doc in relevant_docs)
final_prompt = f"""Answer the question using the context below. If the answer is not in the context, say "I don't know".

Context:
{context}

Question:
{query}
"""

client = InferenceClient(
    provider="novita",
    api_key=os.environ["HF_TOKEN"],
)

completion = client.chat.completions.create(
    model="Qwen/Qwen3-30B-A3B",
    messages=[
        {
            "role": "user",
            "content": final_prompt
        }
    ],
)

print(completion.choices[0].message)

ChatCompletionOutputMessage(role='assistant', content='<think>\nOkay, let\'s see. The user is asking, "What is labor for Arendt?" based on the provided context. I need to look through the excerpts from Hannah Arendt\'s "The Human Condition" to find her definition of labor.\n\nFirst, the context mentions that Arendt distinguishes labor from work and action. It says, "Arendt on how labor (not work or action) are the dominant activity..." So she\'s differentiating between these three terms. Then there\'s a part about the "theoretical glorification of labor" leading to society becoming a "laboring society." \n\nAnother key point is the quote: "The blessing of life as a whole, inherent in labor, can never be found in work..." This suggests that labor is about the basic necessities of life. Also, the text says labor is tied to the "bondage to necessity" and that it\'s about "producing the means of subsistence." \n\nThe passage also talks about how labor is connected to effort and gratificati